In [1]:
"""
This script applies the RISE watertable fluctuation method (e.g., Delin et al., 2007) to estimate 
recharge from synthetic hydrographs. The method is implemented on a series of generated recharge events, and 
recharge estimates are compared to known values. The output is the relative error (%) of the 
recharge estimate, calculated as described in Becke et al. (2025).
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Print version info
print(sys.version)
print(f'numpy version: {np.__version__}')

# Define paths
FOLDER_NAME = "RISE"
BASE_DIR = Path.home() / "workspace" / "WTF" / "OutputFiles" 
OUTPUT_DIR = BASE_DIR / FOLDER_NAME
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

INPUT_DIR = BASE_DIR / "data" 

# Define time array and recharge event windows
time = np.linspace(0, 400., 401)

event_windows = {
    1: {"fit": (30, 90), "extend": (30, 120), "rise": (90, 120)},
    2: {"fit": (120, 210), "extend": (120, 240), "rise": (210, 240)},
    3: {"fit": (240, 360), "extend": (240, 390), "rise": (360, 390)},
}

def compute_RISE_errors(itr, event_id, time, fit_range, extend_range, rise_range):
    x_fit = time[fit_range[0]:fit_range[1] + 1]
    x_extend = time[extend_range[0]:extend_range[1] + 1]
    x_rise = time[rise_range[0]:rise_range[1] + 1]

    errors = []

    for j in range(1, itr + 1):
        try:
            wl_file = INPUT_DIR / f"hydro_{j:04d}.csv"
            var_file = INPUT_DIR / f"var_{j:04d}.csv"

            wl = pd.read_csv(wl_file, header=None).to_numpy().flatten()
            var = pd.read_csv(var_file, header=None).to_numpy()

            wl_rise = wl[rise_range[0]:rise_range[1] + 1]
            delta_h = np.diff(wl_rise)
            delta_h_total = np.sum(delta_h)

            delta_t = x_extend[-1] - x_fit[-1]
            Sy = var[3][0]
            true_rech = var[0][0]
            rech_est = (delta_h_total / delta_t) * Sy
            rech_error = ((rech_est - true_rech) / true_rech) * 100

            errors.append(rech_error)

            print(f"[RE{event_id}] File {j:04d} | Δh_total = {delta_h_total:.4f}, Recharge = {rech_est:.4f}, Error = {rech_error:.2f}%")

        except Exception as e:
            print(f"[RE{event_id}] Hydrograph {j:04d} | Error: {e}")
            continue

    return errors

# --- Run for all events ---
for event_id, params in event_windows.items():
    errors = compute_RISE_errors(
        itr=1000,
        event_id=event_id,
        time=time,
        fit_range=params["fit"],
        extend_range=params["extend"],
        rise_range=params["rise"]
    )

    output_file = OUTPUT_DIR / f"RE{event_id}_RISE_Rerror.csv"
    np.savetxt(output_file, errors, delimiter=',', header=f'RISE_Rerror_Event{event_id}', comments='')
    print(f"Saved errors for Event {event_id} to: {output_file}")


3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.24.3
[RE1] File 0001 | Δh_total = 0.0976, Recharge = 0.0008, Error = -18.67%
[RE1] File 0002 | Δh_total = 1.5140, Recharge = 0.0131, Error = -6.28%
[RE1] File 0003 | Δh_total = 0.4696, Recharge = 0.0038, Error = -21.74%
[RE1] File 0004 | Δh_total = 0.8738, Recharge = 0.0044, Error = -67.88%
[RE1] File 0005 | Δh_total = 0.8313, Recharge = 0.0055, Error = -29.85%
[RE1] File 0006 | Δh_total = 9.8665, Recharge = 0.0099, Error = -18.46%
[RE1] File 0007 | Δh_total = 2.6193, Recharge = 0.0114, Error = -1.30%
[RE1] File 0008 | Δh_total = 0.1788, Recharge = 0.0007, Error = -34.45%
[RE1] File 0009 | Δh_total = 1.7825, Recharge = 0.0125, Error = -8.92%
[RE1] File 0010 | Δh_total = 1.1908, Recharge = 0.0107, Error = -9.94%
Saved errors for Event 1 to: C:\Users\beck0213\workspace\WTF\OutputFiles\GITHUB_CHECK\RISE\RE1_RISE_Rerror.csv
[RE2] File 0001 | Δh_total = 0.0977, Recharge = 0.0008,